# Notebook
This is just a way of making it easier to go through the audio features, and share any work that I've been doing. This is done by Nicolas

This will probably be a bit messy, but it exists and will be commented and that's what matters

In [1]:
%pip install -U sentence-transformers

  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Using cached transformers-4.25.1-py3-none-any.whl (5.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 MB 1.4 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 2.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.7 MB/s eta 0:00:00a 0:00:01
  Using cached huggingface_hub-0.11.1-py3-none-any.whl (182 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 1.9 MB/s eta 0:00:0000:0100:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=dbcf5200a639005e84ae58ceddede63009f4439560a20e9eb5aa05423e8e5c37
  Stored in directory: /Users/nicolasperez/Library/Caches/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

from text_features import get_sentence_similarity

df = pd.read_csv("../out/Bed002_segments_final_sorted.csv", sep=';')
text_features = get_sentence_similarity(df, "Text")
audio_features = pd.read_csv("../out/Bed002_audio_features_of_segments.csv", sep=';')

df_master = audio_features.merge(text_features, left_on='segID', right_on='id', how='inner')

df_master.head()

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

: 

: 